In [1]:
import src.spectral_utils as su
import src.graph_utils as gu
from src import db_helpers
from src.simgraph import SimGraph, from_file
from src import localization

import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from sklearn.cluster import SpectralClustering
from tqdm import tqdm

In [2]:
def dbScorer_from_simgraph_masks(dir_simgraph,dir_mask,mask_name_parser,mask_loader,pixel_predictor,info_dict):
    glist = db_helpers.dir_to_graphlist(dir_simgraph)
    outlist = []
    for gg in tqdm(glist):
        sg = gg[1]
        f_im, f_mask = mask_name_parser(gg)
        M = mask_loader(dir_mask+f_mask)
        pixpred = pixel_predictor(sg,M.shape)
        outlist.append((pixpred,M))
    scorer = localization.dbScorer(outlist,info_dict)
    return scorer

In [3]:
def pixpred_spectral_cluster(sg, mask_shape, assign_labels='discretize',laplacian_type = None,
                             smoothing='Gaussian',smoothing_window=32,normalization=True):
    
    mat = (sg.mat.transpose()+sg.mat)*0.5
    L = su.laplacian(mat,laplacian_type=laplacian_type,degree_axis=1)
    prediction = su.spectral_cluster(L)
    pixpred = localization.pixel_pred_from_patch_pred(prediction,sg.inds,sg.patch_size,
                                                    mask_shape,
                                                    smoothing=smoothing,
                                                    smoothing_window=smoothing_window,
                                                    normalization=normalization)
    return pixpred

def pixpred_fg_modularity(sg, mask_shape, threshold, cluster_weighted=True, k=2,
                          smoothing='Gaussian',smoothing_window=32,normalization=True):
    
    g = gu.adj_to_graph(graph_utils.sym_mat(sg.mat),threshold=threshold)
    com,_,_ = gu.cluster_fastgreedy(g,weighted=cluster_weighted,n=k)
    prediction = np.array(com.membership)>0

    pixpred = localization.pixel_pred_from_patch_pred(prediction,sg.inds,sg.patch_size,
                                                    mask_shape,
                                                    smoothing=smoothing,
                                                    smoothing_window=smoothing_window,
                                                    normalization=normalization)
    return pixpred

def pixpred_spectral_cluster_sklearn(sg, mask_shape, assign_labels='discretize',
                                     smoothing='Gaussian',smoothing_window=32,normalization=True):
    
    sc = SpectralClustering(2, affinity='precomputed', n_init=100, assign_labels=assign_labels)
    sc.fit((sg.mat.transpose()+sg.mat)*0.5)
    prediction = sc.labels_
    pixpred = localization.pixel_pred_from_patch_pred(prediction,sg.inds,sg.patch_size,
                                                    mask_shape,
                                                    smoothing=smoothing,
                                                    smoothing_window=smoothing_window,
                                                    normalization=normalization)
    return pixpred

In [4]:
def disp_score_report(scorer):
    mcc_im = scorer.score_best_thresh_per_image('mcc')[0]
    f1_im = scorer.score_best_thresh_per_image('f1')[0]
    mcc_db = scorer.score_best_thresh_for_db_imavg('mcc')[0]
    f1_db = scorer.score_best_thresh_for_db_imavg('f1')[0]
    auc = scorer.score_auc_pix_total()[0]

    str1 = '{}\n'.format(str(scorer.info))
    str2 = 'mcc_im:{}, f1_im:{}\n'.format(mcc_im,f1_im)
    str3 = 'mcc_db:{}, f1_db:{}\n'.format(mcc_db,f1_db)
    str4 = 'auc:{},\n'.format(auc)
 
    print(str1)
    print(str2)
    print(str3)
    print(str4)

In [5]:
#CARVALHO
f_sg_carvalho = './simg_saves/cam_128/96/carvalho_tampered/'
f_mask_carvalho = 'tampering-databases/carvalho_masks/'

pixel_predictor = lambda sg, mask_shape: pixpred_spectral_cluster(sg,mask_shape)
mask_name_parser = lambda gg: db_helpers.simg_to_carvalho_names(gg[0])
mask_loader = lambda mask_path: localization.load_carvalho_mask(mask_path)
info_dict = {}
info_dict['db']='Carvalho'
scorer = dbScorer_from_simgraph_masks(f_sg_carvalho,f_mask_carvalho,mask_name_parser,mask_loader,pixel_predictor,info_dict)

#these values should match the "Spectral Clustering" row, "Carvalho" column in tables III-VI 
disp_score_report(scorer)

Scoring DB:   0%|          | 0/100 [00:00<?, ?it/s]/home/owen/forensic-graph/src/localization.py:474: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp/(tp+fp)
Calculating ROC: 100%|██████████| 101/101 [00:34<00:00,  2.93it/s]

{'db': 'Carvalho'}

mcc_im:0.8045099464920409, f1_im:0.8260877147059797

mcc_db:0.7460828558229096, f1_db:0.7674190849078905

auc:0.8912492829842968,

